<a href="https://colab.research.google.com/github/Elwing-Chou/tibaml1027/blob/main/res.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.utils import plot_model
model = ResNet50V2(include_top=False, input_shape=(224, 224, 3))
plot_model(model, to_file="res.png", show_shapes=True)